In [172]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [173]:
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import BayesianRidge, Ridge, Lasso
import xgboost as xg
import catboost as ctb
import lightgbm as lgbm
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score,train_test_split
import seaborn as sn
from xgboost import plot_importance
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2
from numpy import sort

In [174]:
train = pd.read_csv('../input/jobathon-august-2022/train_F3fUq2S.csv')
test = pd.read_csv('../input/jobathon-august-2022/test_Bk2wfZ3.csv')
sample = pd.read_csv('../input/jobathon-august-2022/sample_submission_LJ2N3ZQ.csv')

In [175]:
train.drop(['campaign_id', 'is_timer'], axis = 1, inplace = True)
test.drop(['campaign_id', 'is_timer'], axis = 1, inplace = True)

In [176]:
tr = pd.get_dummies(train, columns = ['times_of_day'], drop_first = True)
te = pd.get_dummies(test, columns = ['times_of_day'], drop_first = True)

In [177]:
tr['sub_body_ratio'] = tr['subject_len']/tr['body_len']
te['sub_body_ratio'] = te['subject_len']/te['body_len']

tr['body_sub_ratio'] = tr['body_len']/tr['subject_len']
te['body_sub_ratio'] = te['body_len']/te['subject_len']

# tr['body_paragraph_ratio'] = tr['body_len']/tr['mean_paragraph_len']
# te['body_paragraph_ratio'] = te['body_len']/te['mean_paragraph_len']

tr['paragraph_CTA_len_ratio'] = tr['mean_paragraph_len']/(tr['mean_CTA_len']+1)
te['paragraph_CTA_len_ratio'] = te['mean_paragraph_len']/(1+te['mean_CTA_len'])

# tr['subj_len_CTA_no'] = tr['subject_len']/(1+ tr['no_of_CTA'])
# te['subj_len_CTA_no'] = te['subject_len']/(1+ te['no_of_CTA'])

tr['body_len_CTA_no'] = tr['body_len']/(1+ tr['no_of_CTA'])
te['body_len_CTA_no'] = te['body_len']/(1+ te['no_of_CTA'])


# tr['cta_mean_ratio'] = tr['no_of_CTA']/(tr['mean_CTA_len']+ 1)
# te['cta_mean_ratio'] = te['no_of_CTA']/(te['mean_CTA_len']+ 1)

In [178]:
tr = np.sqrt(tr)
te = np.sqrt(te)

In [179]:
X = tr.drop('click_rate', axis = 1)
y = tr['click_rate']

In [180]:
y_bins = pd.qcut(tr['click_rate'], 5, labels=[1,2,3,4,5]).astype(int)

In [181]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y_bins)

In [182]:
model = xg.XGBRegressor(random_state=42)
model.fit(X_train, y_train)

In [183]:
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)
y_pred = y_pred**2
y_pred_train = y_pred_train**2
print(r2_score(y_train**2,y_pred_train))
print(r2_score(y_test**2, y_pred))

In [184]:
pred = model.predict(te)
sample['click_rate'] = pred ** 2

In [185]:
scores = cross_val_score(model, X_train, y_train, cv=5, scoring = 'r2')
print(scores)
print(np.mean(scores))

In [186]:
sample.to_csv('xgb_sqrt_transform_ratio_features.csv', index = False)